## Accessing the Run1.1p Coadd data using the Data Butler

This notebook shows how to read in the coadd catalog data for an entire tract using the data butler and construct a pandas dataframe with the columns needed to apply the HSC weak lensing cuts.

This is based on [Mandelbaum et al. 2017](https://arxiv.org/abs/1705.06745) using cuts given by [Francois in Slack](https://lsstc.slack.com/archives/C77DDKZHR/p1525197444000687).
We have a [notebook that performs a similar analysis using GCR](https://github.com/LSSTDESC/DC2_Repo/blob/master/Notebooks/DC2%20Coadd%20Run1.1p%20GCR%20access%20--%20HSC%20selection.ipynb).  See also [the more general script](https://github.com/LSSTDESC/DC2_Repo/blob/master/scripts/merge_tract_cat.py) for extracting the coadd data from a tract into an hdf5 file.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lsst.daf.persistence as dp

In [ ]:
def read_tract(butler, tract, filter_='i', num_patches=None):
    """
    Read in the coadd catalog data from the specified tract, looping over patches and concatenating
    into a pandas data frame.  The `merged` coadd catalog is supplemented with the CModel fluxes from
    the `forced_src` catalog for the desired filter.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Data butler that points to the data repository of interest, e.g., the DRP output repository
        for the Run1.1p data.
    tract: lsst.skymap.tractInfo.TractInfo
        The object containing the tract information as obtained from a skyMap object, e.g., via 
        `tract = skymap[tractId]` where `tractId` is the tract number.
    filter_: str ['i']
        Filter to use for extracting the `forced_src` fluxes.
    num_patches: int [None]
        Number of patches to consider.  If None, then use all of the available patches in the tract.

    Returns
    -------
    pandas.DataFrame: This will contain the merged coadd data with the band-specific columns.
    """
    if num_patches is None:
        num_patches = len(tract)
    df_list = []
    i = 0
    for patch in tract:
        patchId = '%d,%d' % patch.getIndex()
        dataId = dict(filter=filter_, tract=tract.getId(), patch=patchId)

        # Patches for which there is no data will raise a `NoResults` exception when requested
        # from the butler.  We catch those exceptions, note the missing data, and 
        # continue looping over the patches.
        try:
            forced = butler.get('deepCoadd_forced_src', dataId)
            calib = butler.get('deepCoadd_calexp_calib', dataId)
            calib.setThrowOnNegativeFlux(False)
            merged = butler.get('deepCoadd_ref', dataId)
        except dp.NoResults as eobj:
            print(eobj)
            continue

        # Convert the merged coadd catalog into a pandas DataFrame and add the band-specific
        # quantities from the forced_src coadd catalog as new columns.
        data = merged.asAstropy().to_pandas()
        data[filter_ + '_modelfit_CModel_flux'] = forced['modelfit_CModel_flux']
        data[filter_ + '_modelfit_CModel_fluxSigma'] = forced['modelfit_CModel_fluxSigma']

        # The calib object applies the zero point to get calibrated magnitudes.
        data[filter_ + '_mag_CModel'] = calib.getMagnitude(forced['modelfit_CModel_flux'])
        data[filter_ + '_mag_err_CModel'] = calib.getMagnitude(forced['modelfit_CModel_fluxSigma'])
        data[filter_ + '_modelfit_CModel_SNR'] \
            = forced['modelfit_CModel_flux']/forced['modelfit_CModel_fluxSigma']
        data['ext_shapeHSM_HsmShapeRegauss_abs_e'] \
            = np.hypot(data['ext_shapeHSM_HsmShapeRegauss_e1'],
                       data['ext_shapeHSM_HsmShapeRegauss_e2'])
        df_list.append(data)

        # Print the current patch and number of objects added to the final DataFrame.
        print(i, patchId, len(data))
        i += 1
        if i == num_patches:
            break
    return pd.concat(df_list)

In order to access the DRP outputs, including images and catalog data, we create a data butler and provide the path to the data repository where the Stack pipeline tasks have written their outputs.

In [ ]:
butler = dp.Butler('/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/output')

The `skyMap` object contains the tract and patch information.  Here we pick a tract that lies in the center of the protoDC2 simulation region for Run1.1p.

In [ ]:
skymap = butler.get('deepCoadd_skyMap')
tractId = 4850
filter_ = 'i'
num_patches = None
data = read_tract(butler, skymap[tractId], filter_=filter_, num_patches=num_patches)
print("Number of objects in the coadd catalog:", len(data))

Apply a baseline set of cuts to remove nan's.

In [ ]:
base_mask = ~(np.isnan(data['i_modelfit_CModel_flux'])
         | np.isnan(data['ext_shapeHSM_HsmShapeRegauss_resolution'])
         | np.isnan(data['ext_shapeHSM_HsmShapeRegauss_e1']))
data = data[base_mask]
print("Number of objects after baseline cuts:", len(data))

These are the weak lensing cuts.   The `detect_isPrimary` flag identifies the primary object in the overlap regions between patches, i.e., applying it resolves duplicates from the different analyses of overlapping patches.

In [ ]:
mask = data['detect_isPrimary']
mask &= data['deblend_skipped'] == False
mask &= data['base_PixelFlags_flag_edge'] == False
mask &= data['base_PixelFlags_flag_interpolatedCenter'] == False
mask &= data['base_PixelFlags_flag_saturatedCenter'] == False
mask &= data['base_PixelFlags_flag_crCenter'] == False
mask &= data['base_PixelFlags_flag_bad'] == False
mask &= data['base_PixelFlags_flag_suspectCenter'] == False
mask &= data['base_PixelFlags_flag_clipped'] == False
mask &= data['ext_shapeHSM_HsmShapeRegauss_flag'] == False

# Cut on object properties
mask &= data['i_modelfit_CModel_SNR'] >= 10
mask &= data['ext_shapeHSM_HsmShapeRegauss_resolution'] >= 0.3
mask &= data['ext_shapeHSM_HsmShapeRegauss_abs_e'] < 2
mask &= data['ext_shapeHSM_HsmShapeRegauss_sigma'] <= 0.4
mask &= data['i_mag_CModel'] < 24.5 # !!! Doesnt have exinction correction
mask &= data['base_Blendedness_abs_flux'] < 10**(-0.375)

These are plots of galaxy properties with the above cuts applied. See Fig. 22 in Mandelbaum et al. 2017.

In [ ]:
fig = plt.figure(figsize=(10, 10))
p1 = fig.add_subplot(2, 2, 1)
hist_kwds = dict(bins=100, histtype='step')
plt.hist(data[filter_ + '_mag_CModel'][mask], **hist_kwds);
plt.xlabel('CModel {}-mag'.format(filter_))
p2 = fig.add_subplot(2, 2, 2)
plt.hist(data[filter_ + '_modelfit_CModel_SNR'][mask], range=(0, 100), **hist_kwds);
plt.xlabel('CModel {}-band S/N'.format(filter_))
p3 = fig.add_subplot(2, 2, 3)
plt.hist(data['ext_shapeHSM_HsmShapeRegauss_resolution'][mask], **hist_kwds);
plt.xlabel('HSM resolution')
p4 = fig.add_subplot(2, 2, 4)
plt.hist(data['ext_shapeHSM_HsmShapeRegauss_abs_e'][mask], **hist_kwds);
plt.xlabel('HSM |e|');